In [6]:
with open('data/smirnenski.txt') as f:
    content = f.read()

In [22]:
print(content[:10], ' - ', len(content))

Нощта е че  -  21589


In [76]:
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
from os import scandir
tokenizer = TweetTokenizer()

def preprocess_text(content):
    sentences = sent_tokenize(content)
    tokens = []
    for _sent in sentences:
        sent_tokens = tokenizer.tokenize(_sent)
        sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
        tokens += sent_tokens
    return tokens

In [103]:
sorted_tokens = preprocess_text(content)

In [104]:
print(len(sorted_tokens), len(set(sorted_tokens)))

3644 1673


In [105]:
from collections import defaultdict

def calc_bigram_freq(sorted_tokens):
    result = defaultdict(lambda: defaultdict(lambda: 0))

    for f, s in zip(sorted_tokens, sorted_tokens[1:]):
        result[f][s] += 1
        
    return result

In [116]:
bigram_freq = calc_bigram_freq([token for token in sorted_tokens if len(token) > 2])

In [195]:
bigram_freq_dict = {k: dict(v) for k, v in bigram_freq.items()}
list(bigram_freq_dict.items())[50:60]

[('назад', {'назад': 1, 'стиснал': 1}),
 ('учуди', {'зовът': 1}),
 ('синове', {'безумци': 1, 'през': 1}),
 ('развява', {'пред': 1}),
 ('край', {'нас': 1, 'него': 1}),
 ('верига', {'майката-земя': 1}),
 ('безумно', {'крещи': 1, 'смелата': 1}),
 ('съдбоносен', {'плам': 1}),
 ('пламъците', {'вулкана': 1}),
 ('лице', {'срещна': 1})]

In [196]:
sorted(bigram_freq_dict.items(), key=lambda x: max(map(lambda y: y[1], x[1].items())))[-4:]

[('деца',
  {'вий': 1,
   'възрастват': 1,
   'лондон': 1,
   'майката': 3,
   'обидата': 1,
   'старци': 1,
   'шумний': 1}),
 ('елате', {'вий': 3, 'подлеци': 1}),
 ('първи', {'зов': 1, 'май': 5, 'тъмнината': 2}),
 ('майката', {'земя': 5})]

In [134]:
bigram_prob = {f: {s: freq / sum(map(lambda x: x[1], goto.items())) for s, freq in goto.items()} 
               for f, goto in bigram_freq_dict.items()}

In [140]:
list(bigram_prob.items())[10:20]

[('отговарят', {'чрез': 1.0}),
 ('сребросинкави', {'петна': 1.0}),
 ('улицата', {'със': 0.5, 'шумна': 0.5}),
 ('строга', {'този': 1.0}),
 ('хладната', {'гръд': 1.0}),
 ('озарен', {'бледата': 0.5, 'чудеса': 0.5}),
 ('стозвучно', {'загърмя': 1.0}),
 ('мощ',
  {'горите': 0.3333333333333333,
   'заканата': 0.3333333333333333,
   'слепци': 0.3333333333333333}),
 ('друго', {'рухват': 1.0}),
 ('беседваш', {'гостенина': 1.0})]

In [183]:
from random import random, choice, seed as seed_random

def prob_choice(items, key=lambda x: x, seed=None):
    seed_random(seed)

    weights = map(key, items)
    total = sum(weights)
    s = 0

    for i, w in enumerate(weights):
        if s >= random():
            return items[i]
        s += w
    return choice(list(items))
    

def prob_walk(seed_word, mutation, gen_size, bigram_prob, seed=None):
    result = [seed_word]
    for _ in range(gen_size):
        new_word = prob_choice(bigram_prob[result[-1]].items(), key=lambda x: x[1], seed=seed)
        result.append(new_word[0])
        
    return result

In [193]:
generated_tokens = prob_walk('улицата', 1, 20, bigram_prob, 5)

In [192]:
print(" ".join(generated_tokens))

улицата със поглед невинност някогашна сегашният ужас снежния берлин шумний град робите разкъсали студените окови властно който вдига смелите младежи сам


In [205]:
generated_tokens = prob_walk('мощ', 1, 13, bigram_prob, 88)

In [206]:
print(" ".join(generated_tokens))

мощ слепци поведени хищници хиени звънтяха сабли летяха настървени кървавата вечер теменужена виждаш бедните
